#Next Generation Sequence Analysis Tools

##Overview

The analysis of next generation sequence data typically involves the subsequent application of multiple tools and utilities, each with their own specific set of parameters and usage characteristics. Although there are currently many different software packages that can be used at various stages of analysis, and are often dictated by the type of sequencing conducted, the general analytical pipeline can be described as follows:

<img src="ngs_pipeline.png" alt="Pipeline" style="width: 600px;"/>

We will be using sequences generated from an Illumina HiSeq2000 machine today, and as such the rest of this document will pertain to analysis of this type of sequencing data.

**Quality Control**   
The first step of any sequencing project is an assessment of the raw sequence data that were generated. There are a number of simple ways to check your sequences. One makes use of the expectation that each sequence position should have a consistent nucleotide usage across all sequence reads, while another assesses per-base quality scores. Often times Illumina sequence reads need to be trimmed at their ends. 

**Alignment**   
After assessing the quality of the sequence and filtering/trimming as needed, the next step is to align the sequence against a reference. There are many software packages for doing this, and most make use of a Burrows-Wheeler transformation to speed up this process. The resulting alignments are then described in SAM format, which stores various information about the sequences including alignment position and mapping quality. More information can be found here: http://genome.sph.umich.edu/wiki/SAM

**Optimization**   
Once a sequence has been aligned, there are a number of additional steps that should be untaken. Some of these, such as cleaning and sorting the SAM file, are required in order to prepare the file for further software packages. Others are tailored to correct the alignments for artifacts stemming from both machine and sequence-based characteristics such as alignment problems around INDELs and quality scores affected by sequencing cycle.

**Variant Calling**   
The accurate identification of genetic variation, both within and between genomic sequences, is one of the primary goals of next generation sequence analysis. Different software packages achieve this in various ways, and range from identifying a variant from a single sequence (heterozygous, or homozygous but different from reference) to across a population of sequences. Quality scores of individual bases are also typically taken into account to minimize false positive results.

**Variant Interpretation**   
The next step after identifying variation is the interpretation of what potential functional role, if any, that it may have. For single nucleotide variants, this is often an assessment of whether a particular allele would result in an amino acid change (non-synonymous change) or a premature stop codon (non-sense change), if found in the coding region of a gene. Other features can include whether a splice site is created/destroyed, or if a variant falls within a known functional region (e.g. transcription factor binding site).

Today we will be taking a set of genomic sequences through our own pipeline. These sequences were taken from the 1000 Genomes Project (samples NA12891 and NA12892) and were sequenced to 100X coverage on an Illumina HiSeq2000 machine at the Broad Institute. We will also be using the personal genome sequence of Gil Omenn, M.D., Ph.D. (http://www.ccmb.med.umich.edu/omenn), a member of our department, that was generated as part of the initial 100 Pioneers of the 100K Wellness Project (https://www.systemsbiology.org/research/100k-wellness-project/). In the interests of time, we will only be analyzing a small subset of these sequences taken from a 100 Kbp region on chromosome 21. 

##iPython Notebooks
We will be making use of iPython Notebooks for today's lab session. These allow you to both comment and actively run python or shell code sequentially and view the results in the browser itself. Indeed, this is what you are viewing now! Today, we will be running many third-party software platforms and as such will need to prepend a "!" character before our commands so it knows to run it as in the shell. Alternatively, these commands (without the "!") can be copy/pasted into a terminal window and run there instead. We will start by viewing the files that were provided.

In [3]:
! ls

bioinf527_ngs_analysis.ipynb  ref.dict	  ref.fa.sa
dbsnp.vcf		      ref.fa	  ref.rev.1.bt2
ngs_pipeline.png	      ref.fa.amb  ref.rev.2.bt2
ref.1.bt2		      ref.fa.ann  s_3_1_sequence.txt
ref.2.bt2		      ref.fa.bwt  s_3_2_sequence.txt
ref.3.bt2		      ref.fa.fai  s_8_1_sequence.txt
ref.4.bt2		      ref.fa.pac  s_8_2_sequence.txt


These files include a reference genome (ref.fa) and associated alignment indexes for bowtie2 (.bt2) and bwa (.bwt). We also see some sequence files (e.g. s_3_1_sequence.txt) as well as a variance call file for dbsnp.

Let's take a quick look at one of the sequence files to see what it looks like:

In [6]:
! head -20 s_3_1_sequence.txt

@B00EGABXX110201:2:4:14018:71991/1
ACCCCTAAGTCCAGCATGACTGGTGTCCTTACAAAAGAGGAAATTTGGACATAAGCACACAGGGAGAACACCCCCATGTGAATATAAAGGCAGAGATAGGG
+
BABBCDBEDBCCCEDCCDDCEDDBDBDCEDDCCFEFEDEDDEECEEDDDCDCDFECDCDBCEEEDEDEBDACACCDCDCDCEBCCCEBDDBCEDCC1A@?+
@B06PYABXX110322:1:2201:16415:170107/1
AATATCTGGTGTTCTTTGGCTTGTAGAAGCACCACACCTATCTCTGCCTTTATATTCACATGGGGGTGTTCTCCCTGTGTGCTTATGTCCAAATTTCCTCT
+
@B@ABCDCCBDBDDEDDDDDED@BBCCDCACB?ABCBCEACCECECDCEDDCCCDDBDC>CC=ACDCDBDBEBDDEDCDCEEAECCDCCCCDCBCC@CDBD
@B06PYABXX110322:1:1208:14933:193200/1
GAAGAATCTGCTCCATGCCTCTCCCCTAGCTTTTTGGGGTTTGCTGACAATATCTGGTGTTCTTTGGCTTGTAGAAGCACCACACCTATCTCTGCCTTTAT
+
BBDDCECCDCCECCCCDDDEDEDDDDECEDEEDEEDDDECEECEEDACDEDDDCEDECECEDEEDEEEED><<@>FDCDCDCBCCEE/BDDCECBDECD@A
@B06PYABXX110322:1:1108:17016:15265/1
GATAAGGTCATACTGGAGTAGGATGAACCCCTAAGTCCAGCATGACTGGTGTCCTTACAAAAGAGGAAATTTGGACATAAGCACACAGGGAGAACACCCCC
+
BBABEDCACCCCBEDDDEBCEDDCDDFCDDDECFECDDDEDDDEECEDDCDCDDEEDDDFDEEDDECEFCDDDDECDDDFEDDCCCCEDDEEDECDBCBCC
@B06PYABXX110322:2:2106:7

Let's start by assessing the quality of the raw fastq sequence files. Paired-end sequencing data files are typically formatted in an s\_[lane]\_[pair]\_sequence.txt file, where lane indicates the lane in which the sequence was obtained from the flow cell, and pair is an index indicating that the sequences in \_1\_ are paired with sequences in \_2\_. In this directory, lane 3 corresponds to sample NA12891 while lane 8 corresponds to sample NA12892. Check the quality of all sequences at once with the following command:



In [7]:
! /class/local/bin/fastqc s_3_1_sequence.txt s_3_2_sequence.txt s_8_1_sequence.txt s_8_2_sequence.txt

Started analysis of s_3_1_sequence.txt
Approx 5% complete for s_3_1_sequence.txt
Approx 10% complete for s_3_1_sequence.txt
Approx 15% complete for s_3_1_sequence.txt
Approx 20% complete for s_3_1_sequence.txt
Approx 25% complete for s_3_1_sequence.txt
Approx 30% complete for s_3_1_sequence.txt
Approx 35% complete for s_3_1_sequence.txt
Approx 40% complete for s_3_1_sequence.txt
Approx 45% complete for s_3_1_sequence.txt
Approx 50% complete for s_3_1_sequence.txt
Approx 55% complete for s_3_1_sequence.txt
Approx 60% complete for s_3_1_sequence.txt
Approx 65% complete for s_3_1_sequence.txt
Approx 70% complete for s_3_1_sequence.txt
Approx 75% complete for s_3_1_sequence.txt
Approx 80% complete for s_3_1_sequence.txt
Approx 85% complete for s_3_1_sequence.txt
Approx 90% complete for s_3_1_sequence.txt
Approx 95% complete for s_3_1_sequence.txt
Analysis complete for s_3_1_sequence.txt
Started analysis of s_3_2_sequence.txt
Approx 5% complete for s_3_2_sequence.txt
Approx 10% complete for

This will create a directory for each fastq file appended with “\_fastqc”, inside of which is a file named fastqc_data.txt. You can view these files with the more (or less) command from the terminal. Alternatively, you can launch firefox and view the fastqc_report.html file that includes images along with the data. Or, as we will do here, you can view directly in this notebook with a little amount of custom syntax:

In [20]:
from IPython.display import HTML
HTML('<iframe src=./s_3_1_sequence_fastqc.html width=1200 height=700></iframe>')

In [21]:
HTML('<iframe src=./s_3_2_sequence_fastqc.html width=1200 height=700></iframe>')

In [22]:
HTML('<iframe src=./s_8_1_sequence_fastqc.html width=1200 height=700></iframe>')

In [23]:
HTML('<iframe src=./s_8_2_sequence_fastqc.html width=1200 height=700></iframe>')

Do any of the sequences fail the “Basic Statistics”? Which individual modules report a failure in quality?